## Laboratorium 6

imports

In [43]:
import wikipedia
from wikipedia.exceptions import WikipediaException
import pickle
import csv
import numpy as np
import os
import random
import time
import re
from typing import List, Dict
import string
from gensim.parsing.preprocessing import remove_stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from scipy import sparse

[nltk_data] Downloading package stopwords to /home/hiro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hiro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Tekst artukułu jest wyszukiwany i pobierany za pomocą biblioteki wikipedia. Po otrzymaniu danego artykułu jest on zapisywany do folderu ```articles``` za pomocą blbioteki ```pickle```.

Załadownie n-pierwszych artykułów z enwiki dump. (Jest to potrzebne ponieważ plik, który zainstalowałem ma ponad 300 000 tytułów artykułu)

In [2]:
class ArticleLoader:
    def __init__(self, n=10**4, file='articles/enwiki-pages-articles.txt'):
        self.n = n
        self.file = file
        self.enwiki_name = 'enwiki-pages-articles.txt'
        self.titles = []
        self.dict_of_key_words = dict()
    # zadanie 1 - przygotowanie zbioru dokumentów tekstowych
    def get_names_of_n_first_articles(self):
        with open(self.file, 'r',encoding="utf8") as f:
            csv_reader = csv.reader(f, delimiter=':')
            i = 0
            for row in csv_reader:
                if i >= self.n:
                    break

                title_of_article = row[-1]
            
                if '/' in title_of_article:
                    continue

                self.titles.append(title_of_article)
                i += 1

        random.shuffle(self.titles)
    
    def load_and_save_articles(self):
        not_found = 0
        if not self.titles:
            self.get_names_of_n_first_articles()

        for title in self.titles:
            try:
                wiki_page = wikipedia.page(title)
                url = wiki_page.url
                content = wiki_page.content
                dict_of_title = {'content': content, 'url': url}
                pickle.dump(dict_of_title, open('articles/{}'.format(title), 'wb'))
            except WikipediaException:
                not_found += 1
            except Exception as e:
                print("exception ocured {}".format(str(e)))
        print("Titles not found: {}".format(not_found))

    # zadabie 2 - określenie słownika słów kluczowych -> będzie to słwonik z kluczem jako nazwa artykułu, a wartością jako tekst artykułu
    
    # stworzenie nowego słownika dla słów kluczowych
    def create_dict_of_key_words(self):
        i = 0
        # self.dict_of_key_words["6"] = pickle.load(open("articles/{}".format(6), "rb"))

        for title in self.pickle_yeild():
            if title == self.enwiki_name or "key_words" in title: 
                continue

            self.dict_of_key_words[title] = pickle.load(open("articles/{}".format(title), "rb"))
            i+=1
            time.sleep(0.1)


        pickle.dump(self.dict_of_key_words, open('articles/dumped_data/dict_of_key_words_{}'.format(len(self.dict_of_key_words)), 'wb'))
    
    def pickle_yeild(self, path = 'articles'):
        for content in os.listdir(path):
            if os.path.isfile(os.path.join(path, content)):
                yield content

    # wczytanie wcześniej stworzonego słownika dla słów kluczowych
    def load_dict_of_key_words(self, dirpath = 'articles/dumped_data', dumped_file = 'dict_of_key_words_7285'):
        path = os.path.join(dirpath, dumped_file)
        self.dict_of_key_words = pickle.load(open(path, "rb"))




In [3]:
article_handler = ArticleLoader()

In [4]:
article_handler.get_names_of_n_first_articles()

In [5]:
article_handler.load_dict_of_key_words()

In [7]:
print(article_handler.dict_of_key_words[' A New Spelling of My Name']['content'])

Zami: A New Spelling of My Name is a 1982 biomythography by American poet Audre Lorde. It started a new genre that the author calls biomythography, which combines history, biography, and myth. In the text, Lorde writes that "Zami" is "a Carriacou name for women who work together as friends and lovers", noting that Carriacou is the Caribbean island from which her mother immigrated. The name proves fitting: Lorde begins Zami writing that she owes her power and strength to the women in her life, and much of the book is devoted to detailed portraits of other women.


== Plot summary ==
Audre Lorde grows up in Harlem in the 1930s and 1940s, a child of Black West Indian parents. Lorde is legally blind from a very young age, isolating her even further from her surroundings and a family from which she does not receive much warmth or affection. Her two older sisters, Phyllis and Helen, are very close, but are rarely mentioned in Zami and Lorde spends little time with them. Her parents and other

In [67]:
class ArticleSearchEngine(object):
    def __init__(self, articles_key_words : Dict[str, Dict[str, str]]):
        self.articles = articles_key_words
        self.titles = list(self.articles.keys())
        self.occured_words = defaultdict(lambda: 0)
        self.uniq_occured_words = None
        self.ids_of_unique_occured_word = None
        self.parsed_article = dict()
        self.stop_words = set(stopwords.words('english'))
        self.lemmatisation = WordNetLemmatizer()

    def parse_article_text(self, text: str, if_article = True):                     # dzięki if_article możemy się dowiedzieć czy dany artykuł został już wcześniej sprawdzony 
        parse_text = text.lower()                                                   # nie ma sensu w odróżnianiu małych i dużych liter
        parse_text = re.sub('['+string.punctuation+']', '', parse_text).split()     # parsowanie po tekście w celu usunięcia niepotrzebnych znajów np. #$%...
        parsed_text = [word for word in parse_text if word not in self.stop_words]  # usuwanie stop_wordów np. 'ours', 'i' ...

        words_dictonary = defaultdict(lambda: 0)                                    # dla nie powstałego słownika przypisz 0 jako wartość
        words_counter = 0                                                           # liczenie ilości słów
        for word in parsed_text:
            lemma_word = self.lemmatisation.lemmatize(word, pos='v')              # pos = 'v' - verb lemmatisation np. goes -> go ...
            words_dictonary[lemma_word] += 1                                        # dzięki defaultdict mamy pewność, że jak nie istnieje dany klucz to stworzy się nowy z wartością 0
            words_counter += 1
            if if_article:
                self.occured_words[lemma_word] += 1
        return dict(words_dictonary = words_dictonary, words_counter = words_counter)
            

    def article_parser(self):
        for article_title in self.titles:
            self.parsed_article[article_title] = dict()
            self.parsed_article[article_title]['article_content'] = self.parse_article_text(self.articles[article_title]['content'])

        self.uniq_occured_words = list(self.occured_words.keys())
        self.ids_of_unique_occured_word = {self.uniq_occured_words[i]: i for i in range(len(self.uniq_occured_words))}

    def create_bags_of_words(self):
        for article_title in self.titles:
            self.parsed_article[article_title]['bag_of_words'] = self.fill_bag_of_words(self.parsed_article[article_title]['article_content'])
    
    def fill_bag_of_words(self, article_content):
        article_words = article_content['words_dictonary'].keys()

        # tworzenie macierzy rzadkiej -> wymiary: ilość wszystkich unikalnych słów we wszystkich tekstów / 1 -> jeden, ponieważ działamy na jednym artykule w danym momencie
        # dok_matrix -> Dictionary Of Keys based sparse matrix, efficient structure for constructing sparse matrices incrementally
        vector = sparse.dok_matrix(np.zeros((len(self.ids_of_unique_occured_word), 1))) 
        for word_key in article_words:
            vector[self.ids_of_unique_occured_word[word_key], 0] = article_content['words_dictonary'][word_key]

        vector /= article_content['words_counter'] # normalizacja wektora -> dzielenie przez ilość unikalnych słów w artykule
        return sparse.csr_matrix(vector) # csr_matrix -> Compressed Sparse Row matrix

In [66]:
tab = sparse.dok_matrix(np.zeros((10, 1)))
for i in range(10):
    tab[i, 0] = i
print(tab.get((1,0)))

1.0


In [18]:
article_engine = ArticleSearchEngine(article_handler.dict_of_key_words)

In [19]:
article_engine.article_parser()

In [45]:
article_engine.parsed_article[' A New Spelling of My Name']['article_content']['words_counter']

716

In [62]:
content = "Audre Lorde grows up in Harlem in the 1930s and 1940s, a child of Black West Indian parents. Lorde is legally blind from a very young age, isolating her even further from her surroundings and a family from which she does not receive much warmth or affection. Her two older sisters, Phyllis and Helen, are very close, but are rarely mentioned in Zami and Lorde spends little time with them. Her parents and other adults, especially her mother, discipline her harshly for insolence. Lorde"

content = content.lower()
content2 = content.lower()
print(content)
content = re.sub(r'[^\w\s]', '', content)
print(content)
content = re.sub('[0-9]', '', content)
print(content)
content = re.sub(' {2} +',' ', content)
print(content)
content = remove_stopwords(content)
print(content)

# parsowanie stringów
content2 = re.sub('['+string.punctuation+']', '', content2).split()
print(content2)

# usuowanie stopwords
stop_words = set(stopwords.words('english'))
content2re = [word for word in content2 if word not in stop_words]


print(content2re)
print(content.split())
print(len(content.split()))
print(len(content2re))

audre lorde grows up in harlem in the 1930s and 1940s, a child of black west indian parents. lorde is legally blind from a very young age, isolating her even further from her surroundings and a family from which she does not receive much warmth or affection. her two older sisters, phyllis and helen, are very close, but are rarely mentioned in zami and lorde spends little time with them. her parents and other adults, especially her mother, discipline her harshly for insolence. lorde
audre lorde grows up in harlem in the 1930s and 1940s a child of black west indian parents lorde is legally blind from a very young age isolating her even further from her surroundings and a family from which she does not receive much warmth or affection her two older sisters phyllis and helen are very close but are rarely mentioned in zami and lorde spends little time with them her parents and other adults especially her mother discipline her harshly for insolence lorde
audre lorde grows up in harlem in the